In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image = annotated_image,
        landmark_list = face_landmarks_proto,
        connections = solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec = None,
        connection_drawing_spec = solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image = annotated_image,
        landmark_list = face_landmarks_proto,
        connections = solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec = None,
        connection_drawing_spec = solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image = annotated_image,
        landmark_list = face_landmarks_proto,
        connections = solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec = None,
          connection_drawing_spec = solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

2023-08-31 10:13:20.040519: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:


# STEP 1: Import the necessary modules.
import cv2 as cv
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an FaceLandmarker object.
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

cap = cv.VideoCapture(1)

while True:
    ret, frame = cap.read()
    
    key = cv.waitKey(1)
    
    # STEP 3: Load the input image.
    #image = mp.Image(frame)
    
    # STEP 4: Detect face landmarks from the input image.
    detection_result = detector.detect(frame)
    
    # STEP 5: Process the detection result. In this case, visualize it.
    annotated_image = draw_landmarks_on_image(frame.numpy_view(), detection_result)
    
    if key == ord('q'):
        break

    cv.imshow(cv.cvtColor(annotated_image, cv.COLOR_RGB2BGR))

cap.release()
cv.destroyAllWindows()

W20230831 10:30:47.959283 208058 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.


ValueError: Please provide 'image_format' with 'data'.

In [12]:
import cv2 as cv

cap = cv.VideoCapture(1)

while True:
    ret, frame = cap.read()
    
    key = cv.waitKey(1)
    #frame = draw_landmarks_on_image(frame)
    if key == ord('q'):
        break

    cv.imshow('Interface', frame)
    
cap.release()
cv.destroyAllWindows()